In [1]:
from pathlib import Path
import sys

project_root = Path.cwd().resolve().parent  # если ноутбук в experiments/
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

lib_path = str(Path.cwd() / "install" / "lib")
if lib_path not in sys.path:
    sys.path.insert(0, lib_path)


In [ ]:
from src.app_types import FramePointCloud, Intrinsics
import numpy as np
from pyorbbecsdk import Config, PointCloudFilter, OBFormat, Frame
from pyorbbecsdk import OBSensorType
from pyorbbecsdk import Pipeline


class OrbbecSource:
    def __init__(self):
        self.config = Config()
        self.pipeline = Pipeline()
        try:
            profile_list = self.pipeline.get_stream_profile_list(OBSensorType.DEPTH_SENSOR)
            assert profile_list is not None
            depth_profile = profile_list.get_default_video_stream_profile()
            assert depth_profile is not None
            print("depth profile: ", depth_profile)
            self.depth_intrinsics = depth_profile.get_intrinsic()    
            self.config.enable_stream(depth_profile)
        except Exception as e:
            print(e)
            return
        self.pipeline.start(self.config)
        self.point_cloud_filter = PointCloudFilter()
        self.point_cloud_filter.set_create_point_format(OBFormat.POINT)

    def read(self):
        frames = self.pipeline.wait_for_frames(500)
        if frames is None:
            raise Exception("Frame was not obtained")
        point_cloud_frame = self.point_cloud_filter.process(frames)
        
        intrinsics = Intrinsics(self.depth_intrinsics.fx,
                                self.depth_intrinsics.fy,
                                self.depth_intrinsics.cx,
                                self.depth_intrinsics.cy,
                                self.depth_intrinsics.width,
                                self.depth_intrinsics.height)
        
        return FramePointCloud(cloud=point_cloud_frame,
                          intrinsics=intrinsics, depth_scale=1.0)
        
    

load extensions from /home/alexey/miniconda3/envs/mainenv/lib/python3.12/site-packages/extensions


In [3]:
source = OrbbecSource()


[02/04 11:21:19.305099][error][227984][UsbEnumeratorLibusb.cpp:157] Invalid descriptor index: 0
[02/04 11:21:19.305119][error][227984][UsbEnumeratorLibusb.cpp:403] Failed to query USB device serial number
[02/04 11:21:19.307085][error][227984][UsbEnumeratorLibusb.cpp:157] Invalid descriptor index: 0
[02/04 11:21:19.307092][error][227984][UsbEnumeratorLibusb.cpp:419] Failed to query USB device interface name


RuntimeError: No device found

In [ ]:
frame = source.read()
print("Point cloud shape:", frame.cloud.shape)